# Variant annotation demonstration

## Setup

The 'variants' package defines dependencies for this session.

In [72]:
suppressPackageStartupMessages({
library('variants')
})

## A VCF for a personalized genome

We work with Complete Genomics public sequencing of DNA from a Coriell cell line from the CEPH/HapMap cohort.

In [73]:
## --------------------------------------------------------------------------
file <- system.file("vcf", "NA06985_17.vcf.gz", package = "cgdv17")

## --------------------------------------------------------------------------
hdr <- scanVcfHeader(file)

info(hdr)

DataFrame with 3 rows and 3 columns
        Number        Type                 Description
   <character> <character>                 <character>
NS           1     Integer Number of Samples With Data
DP           1     Integer                 Total Depth
DB           0        Flag dbSNP membership, build 131

In [74]:
geno(hdr)

DataFrame with 12 rows and 3 columns
            Number        Type                         Description
       <character> <character>                         <character>
GT               1      String                            Genotype
GQ               1     Integer                    Genotype Quality
DP               1     Integer                          Read Depth
HDP              2     Integer                Haplotype Read Depth
HQ               2     Integer                   Haplotype Quality
...            ...         ...                                 ...
mRNA             .      String                     Overlaping mRNA
rmsk             .      String                  Overlaping Repeats
segDup           .      String Overlaping segmentation duplication
rCov             1       Float                   relative Coverage
cPd              1      String                called Ploidy(level)

In [75]:
## --------------------------------------------------------------------------
meta(hdr)$META

NULL

## Defining the query gene, gathering coordinates

In [76]:
#genesym <- c("TRPV1", "TRPV2", "TRPV3")
genesym <- c("ORMDL3")
geneid <- select(org.Hs.eg.db, keys=genesym, keytype="SYMBOL",
		 columns="ENTREZID")
geneid

'select()' returned 1:1 mapping between keys and columns


SYMBOL,ENTREZID
ORMDL3,94103


In [77]:
## --------------------------------------------------------------------------
txdb <- TxDb.Hsapiens.UCSC.hg19.knownGene
txdb

TxDb object:
# Db type: TxDb
# Supporting package: GenomicFeatures
# Data source: UCSC
# Genome: hg19
# Organism: Homo sapiens
# Taxonomy ID: 9606
# UCSC Table: knownGene
# Resource URL: http://genome.ucsc.edu/
# Type of Gene ID: Entrez Gene ID
# Full dataset: yes
# miRBase build ID: GRCh37
# transcript_nrow: 82960
# exon_nrow: 289969
# cds_nrow: 237533
# Db created by: GenomicFeatures package from Bioconductor
# Creation time: 2015-10-07 18:11:28 +0000 (Wed, 07 Oct 2015)
# GenomicFeatures version at creation time: 1.21.30
# RSQLite version at creation time: 1.0.0
# DBSCHEMAVERSION: 1.1

In [78]:
## --------------------------------------------------------------------------
#txdb <- renameSeqlevels(txdb, gsub("chr", "", seqlevels(txdb)))
#txdb <- keepSeqlevels(txdb, "17")
library(GenomeInfoDb)
seqlevelsStyle(txdb) = "NCBI"

## --------------------------------------------------------------------------
txbygene = transcriptsBy(txdb, "gene")

## --------------------------------------------------------------------------
gnrng <- unlist(range(txbygene[geneid$ENTREZID]), use.names=FALSE)
names(gnrng) <- geneid$SYMBOL
gnrng

GRanges object with 1 range and 0 metadata columns:
         seqnames            ranges strand
            <Rle>         <IRanges>  <Rle>
  ORMDL3       17 38077296-38083884      -
  -------
  seqinfo: 25 sequences (1 circular) from hg19 genome

## Import and inspect a slice of VCF

In [79]:
## --------------------------------------------------------------------------
param <- ScanVcfParam(which = gnrng+20000, info = "DP", geno = c("GT", "cPd"))
param

class: ScanVcfParam 
vcfWhich: 25 elements
vcfFixed: character() [All] 
vcfInfo: DP 
vcfGeno: GT cPd 
vcfSamples:  

In [80]:
## Extract the TRPV ranges from the VCF file
vcf <- readVcf(file, "hg19", param)
## Inspect the VCF object with the 'fixed', 'info' and 'geno' accessors
vcf

class: CollapsedVCF 
dim: 130 1 
rowRanges(vcf):
  GRanges with 5 metadata columns: paramRangeID, REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 1 column: DP
info(header(vcf)):
      Number Type    Description
   DP 1      Integer Total Depth
geno(vcf):
  SimpleList of length 2: GT, cPd
geno(header(vcf)):
       Number Type   Description         
   GT  1      String Genotype            
   cPd 1      String called Ploidy(level)

## Analyze the discovered variants

In [81]:
seqlevels(vcf)

[1] "1"  "2"  "3"  "4"  "5"  "6"  "7"  "8"  "9"  "10" "11" "12" "13" "14" "15"
[16] "16" "17" "18" "19" "20" "21" "22" "X"  "Y"  "M"

In [82]:
txdb = keepStandardChromosomes(txdb)
seqlevelsStyle(txdb) = "NCBI"
seqlevels(txdb)

[1] "1"  "2"  "3"  "4"  "5"  "6"  "7"  "8"  "9"  "10" "11" "12" "13" "14" "15"
[16] "16" "17" "18" "19" "20" "21" "22" "X"  "Y"  "MT"

In [83]:
seqlevels(vcf)[25] = "MT"
seqlevels(vcf)

[1] "1"  "2"  "3"  "4"  "5"  "6"  "7"  "8"  "9"  "10" "11" "12" "13" "14" "15"
[16] "16" "17" "18" "19" "20" "21" "22" "X"  "Y"  "MT"

In [84]:
all <- locateVariants(vcf, txdb, AllVariants())
all

Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 2 out-of-bound ranges located on sequence
  61097. Note that ranges located on a sequence whose length is unknown
  (NA) or on a circular sequence are not considered out-of-bound (use
  seqlengths() and isCircular() to get the lengths and circularity flags
  of the underlying sequences). You can use trim() to trim these ranges.
  See ?`trim,GenomicRanges-method` for more information.”'select()' returned many:1 mapping between keys and columns
'select()' returned many:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned many:1 mapping between keys and columns
'select()' returned many:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns


GRanges object with 542 ranges and 9 metadata columns:
                      seqnames            ranges strand |   LOCATION  LOCSTART
                         <Rle>         <IRanges>  <Rle> |   <factor> <integer>
   17:38057800_TGTT/T       17 38057800-38057803      * | intergenic      <NA>
           rs11657449       17          38057841      * | intergenic      <NA>
    17:38060534_GGC/.       17 38060534-38060536      * | intergenic      <NA>
  17:38060609_ATAAA/.       17 38060609-38060613      * | intergenic      <NA>
      17:38060627_A/.       17          38060627      * | intergenic      <NA>
                  ...      ...               ...    ... .        ...       ...
            rs4065986       17          38102641      * | intergenic      <NA>
            rs3893044       17          38103016      * | intergenic      <NA>
           rs62068170       17          38103210      * | intergenic      <NA>
           rs62068171       17          38103242      * | intergenic      <N

In [85]:
## --------------------------------------------------------------------------
## Did any variants match more than one gene?
table(sapply(split(mcols(all)$GENEID, mcols(all)$QUERYID),
      function(x) length(unique(x)) > 1))
             


FALSE  TRUE 
  112    18 

In [86]:
idx <- sapply(split(mcols(all)$QUERYID, mcols(all)$GENEID), unique)
sapply(idx, length)
             

100505591     55876     94103 
       10        56        18

In [87]:
## Summarize variant location by gene:
sapply(names(idx),
    function(nm) {
	d <- all[mcols(all)$GENEID %in% nm, c("QUERYID", "LOCATION")]
	table(mcols(d)$LOCATION[duplicated(d) == FALSE])
    })

,100505591,55876,94103
spliceSite,0,0,0
intron,3,25,3
fiveUTR,0,7,0
threeUTR,0,1,1
coding,2,3,1
intergenic,0,0,0
promoter,5,20,13


## Variant impact assessment

In [88]:
## --------------------------------------------------------------------------
seqlevelsStyle(vcf) <- "UCSC"
seqlevelsStyle(txdb) <- "UCSC"
aa <- predictCoding(vcf, txdb, Hsapiens)
aa

Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 2 out-of-bound ranges located on sequence
  61097. Note that ranges located on a sequence whose length is unknown
  (NA) or on a circular sequence are not considered out-of-bound (use
  seqlengths() and isCircular() to get the lengths and circularity flags
  of the underlying sequences). You can use trim() to trim these ranges.
  See ?`trim,GenomicRanges-method` for more information.”Warning message in .predictCodingGRangesList(query, cache[["cdsbytx"]], seqSource, :
“records with missing 'varAllele' were ignored”

GRanges object with 25 ranges and 17 metadata columns:
                      seqnames            ranges strand | paramRangeID
                         <Rle>         <IRanges>  <Rle> |     <factor>
      17:38062111_T/.    chr17          38062111      - |       ORMDL3
      17:38062111_T/.    chr17          38062111      - |       ORMDL3
      17:38062111_T/.    chr17          38062111      - |       ORMDL3
      17:38062111_T/.    chr17          38062111      - |       ORMDL3
      17:38062111_T/.    chr17          38062111      - |       ORMDL3
                  ...      ...               ...    ... .          ...
            rs2305479    chr17          38062217      - |       ORMDL3
      17:38080385_G/.    chr17          38080385      - |       ORMDL3
      17:38080385_G/.    chr17          38080385      - |       ORMDL3
      17:38100278_G/.    chr17          38100278      + |       ORMDL3
  17:38100751_CAGGA/.    chr17 38100751-38100755      + |       ORMDL3
                      

In [89]:
## --------------------------------------------------------------------------
## Did any variants match more than one gene?
table(sapply(split(mcols(aa)$GENEID, mcols(aa)$QUERYID),
	function(x) length(unique(x)) > 1))

## Summarize the number of variants by gene:
idx <- sapply(split(mcols(aa)$QUERYID, mcols(aa)$GENEID, drop=TRUE), unique)
sapply(idx, length)
             


FALSE 
    6 

100505591     55876     94103 
        2         3         1

In [90]:
## Summarize variant consequence by gene:
sapply(names(idx),
       function(nm) {
	   d <- aa[mcols(aa)$GENEID %in% nm, c("QUERYID","CONSEQUENCE")]
	   table(mcols(d)$CONSEQUENCE[duplicated(d) == FALSE])
       })

sessionInfo()

,100505591,55876,94103
nonsynonymous,0,2,0
not translated,2,1,1


R Under development (unstable) (2018-11-16 r75612)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS Sierra 10.12.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.6/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.6/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats4    parallel  stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] variants_1.7.1                         
 [2] PolyPhen.Hsapiens.dbSNP131_1.0.2       
 [3] RSQLite_2.1.1                          
 [4] BSgenome.Hsapiens.UCSC.hg19_1.4.0      
 [5] BSgenome_1.51.0                        
 [6] rtracklayer_1.43.1                     
 [7] TxDb.Hsapiens.UCSC.hg19.knownGene_3.2.2
 [8] GenomicFeatures_1.35.7                 
 [9] org.Hs.eg.db_3.7.0                     
[10] AnnotationDbi_1.45.0                   
[1